# Toronto Neighborhoods Assignment

## Part 1 - Wiki Page Scraping

In [96]:
!pip install lxml

import lxml
from lxml import html
import requests
import pandas as pd
import numpy as np

f=open("./credentials.txt","r")
lines=f.readlines()
API_key=lines[0]
f.close()
 

### scraping of Wiki page into a dataframe

In [97]:
df=pd.read_html(requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text)[0]

### Changing column names to fit assignment requirements

In [98]:
df.rename(columns={'Postal Code': 'PostalCode','Neighbourhood': 'Neighborhood'}, inplace=True)

### Deleting non assigned Borough lines

In [99]:
df=df[df['Borough']!='Not assigned']

### Nothing to be done here, there is no such case:
We were asked:  "More than one neighborhood can exist in one postal code area [...]"

Print "warning" in case there was duplicates or if not print "Nothing to be done":

In [100]:
data_is_clean=True
for i in df['PostalCode'].value_counts()!=1:
    if i==True:
        print("Warning, you need to clean PostalCode duplicates!")
        data_is_clean=False
if data_is_clean : print("Nothing to be done")

Nothing to be done


### Nothing to be done here either, there is no such case:
We were asked: "If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough."

In [101]:
df[df['Neighborhood']=="Not Assigned"]

,PostalCode,Borough,Neighborhood


### (But if so, we should run this instruction:)

In [102]:
df=df[df['Borough']!='Not assigned']
df=df.reset_index(drop=True)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Result Dataframe Shape

In [103]:
df.shape

(103, 3)

## End of Part 1
## Part 2 - Add Geospatial data

### Installing googlemaps API

In [104]:
!pip install -U googlemaps

Requirement already up-to-date: googlemaps in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (4.4.2)


### Testing googlemaps API call 

In [105]:
import googlemaps
from datetime import datetime
gmaps = googlemaps.Client(key=API_key)

# Geocoding an address
geocode_result = gmaps.geocode('{}, Toronto, Ontario'.format("M4A"))
geocode_result

[{'address_components': [{'long_name': 'M4A',
    'short_name': 'M4A',
    'types': ['postal_code', 'postal_code_prefix']},
   {'long_name': 'North York',
    'short_name': 'North York',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Toronto',
    'short_name': 'Toronto',
    'types': ['locality', 'political']},
   {'long_name': 'Toronto Division',
    'short_name': 'Toronto Division',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Ontario',
    'short_name': 'ON',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Canada',
    'short_name': 'CA',
    'types': ['country', 'political']}],
  'formatted_address': 'North York, ON M4A, Canada',
  'geometry': {'bounds': {'northeast': {'lat': 43.7435851,
     'lng': -79.29866799999999},
    'southwest': {'lat': 43.7151669, 'lng': -79.32717799999999}},
   'location': {'lat': 43.72588229999999, 'lng': -79.3155716},
   'location_type': 'APPROXIMATE',


In [106]:
geocode_result[0]['geometry']['location']['lat']
geocode_result[0]['geometry']['location']['lng']

-79.3155716

In [107]:
import googlemaps
from datetime import datetime
gmaps = googlemaps.Client(key=API_key)

In [108]:
#add lat and lng columns
df["Latitude"] = 0.1
df["Longitude"] = 0.1

In [109]:
#get all geolocs
for code in df['PostalCode']:
    geocode_result = gmaps.geocode('{}, Toronto, Ontario'.format(code))
    df.loc[df['PostalCode'] == code, ['Latitude', 'Longitude']] = (geocode_result[0]['geometry']['location']['lat'],
                                                                   geocode_result[0]['geometry']['location']['lng'])
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## End of Part 2
## Part 3 - 

In [95]:
Exploring and clustering the neighborhoods in Toronto as it was done for NYC in the lab

??? decide to work with only boroughs that contain the word Toronto

TODO :
# to add enough Markdown cells to explain what you decided to do and to report any observations you make.
# to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)



SyntaxError: invalid syntax (<ipython-input-95-17fda385abff>, line 1)